In [1]:
# 1단계: Colab 환경 설정
from google.colab import drive
drive.mount('/content/drive')

# 지리정보 데이터 처리에 필요한 라이브러리를 설치합니다.
# - geopandas: Shapefile과 같은 벡터 데이터를 다루기 위한 라이브러리
# - rasterio: TIF와 같은 래스터(이미지) 데이터를 다루기 위한 라이브러리
!pip install geopandas rasterio



# 2단계: 라이브러리 임포트 및 함수 정의
import os
import rasterio
import geopandas as gpd
import numpy as np
import torch
from rasterio.windows import Window
from rasterio.features import rasterize
from shapely.geometry import box


def convert_to_target_format(label_mask):
    """
    [헬퍼 함수]
    '통합 라벨 마스크'([H, W])를 모델 학습에 필요한 'targets' 딕셔너리 형식으로 변환합니다.
    이 함수는 GIS 데이터(정수 ID)를 딥러닝 모델이 이해할 수 있는 형식(이진 마스크)으로 번역하는 역할을 합니다.
    """
    # 통합 마스크에서 고유한 건물 ID(1, 2, 3...)를 모두 찾아냅니다. 배경 값인 0은 제외합니다.
    instance_ids = torch.unique(label_mask)
    instance_ids = instance_ids[instance_ids != 0]

    # 만약 현재 타일에 건물이 하나도 없다면(ID가 없다면), 학습 시 오류가 나지 않도록
    # 내용이 비어있는 텐서를 가진 딕셔너리를 반환합니다.
    if len(instance_ids) == 0:
        return {'instance_class': torch.tensor([], dtype=torch.long), 'mask': torch.tensor([], dtype=torch.uint8)}

    #'브로드캐스팅'이라는 텐서 연산을 사용하여, 각 건물 ID에 해당하는 픽셀만 1(True)이고
    # 나머지는 0(False)인 개별 마스크들을 효율적으로 생성합니다.
    # 결과적으로 [N, H, W] 크기의 3차원 텐서가 만들어집니다. (N: 건물 개수)
    masks = (label_mask == instance_ids[:, None, None])

    # 현재 프로젝트에서는 '건물'이라는 단일 클래스만 사용하므로,
    # 건물 개수(N)만큼 0으로 채워진 클래스 라벨 텐서를 생성합니다.
    instance_classes = torch.zeros(len(instance_ids), dtype=torch.long)

    # 팀원이 요청한 최종 형식에 맞춰 'instance_class'와 'mask'를 키로 하는 딕셔너리를 구성하여 반환합니다.
    return {'instance_class': instance_classes, 'masks': masks.to(torch.uint8)}


def create_final_dataset_per_file(
    ortho_image_path,
    building_shp_path,
    output_dir,
    tile_size=1024
):
    """
    [RAM/스토리지 문제 해결 최종 버전]
    GIS 데이터를 가공하여, 이미지 타일과 그에 1:1로 대응하는 'targets' 딕셔너리를
    각각 개별 파일로 저장하여 RAM 문제를 해결하는 최종 전처리 함수임

    [중요] 왜 '딕셔너리의 리스트' ([{...}, {...}, ...])를 만들지 않는가?
    ----------------------------------------------------------------
    이 '딕셔너리의 리스트' 형식은 모든 라벨 딕셔너리를 하나의 거대한 리스트 변수에
    담아두었다가 마지막에 한 번에 파일로 저장하는 방식임

    발생했던 문제:
    1. 메모리 초과 (RAM OOM): 'targets' 딕셔너리는 내부에 [N, H, W] 크기의 큰 마스크 텐서를
       포함하여 용량이 매우 큽니다. 수천 개의 딕셔너리를 하나의 리스트에 모두 담으면
       Colab의 제한된 RAM(12GB)을 순식간에 초과하여 커널 재시작 오류가 발생했음
    2. 저장 비효율: 최종적으로 생성되는 단일 리스트 파일의 크기가 매우 커져
       Google Drive의 저장 공간도 빠르게 소진됨

    현재 코드의 해결책:
    이 함수는 한 번에 하나의 타일만 처리하고, 생성된 이미지와 라벨 딕셔너리를 즉시
    개별 파일로 저장함. 이 '파일 단위 저장' 방식은 메모리 사용량을 최소화하여
    Colab 환경에서도 대용량 데이터를 안정적으로 처리함
    ----------------------------------------------------------------
    """
    # 결과물이 저장될 'images'와 'labels_dict' 폴더를 생성합니다.
    img_tile_dir = os.path.join(output_dir, "images")
    lbl_dict_dir = os.path.join(output_dir, "labels_dict")
    os.makedirs(img_tile_dir, exist_ok=True)
    os.makedirs(lbl_dict_dir, exist_ok=True)

    # GeoPandas로 건물 shapefile을, Rasterio로 정사영상(.tif)을 엽니다.
    gdf_buildings = gpd.read_file(building_shp_path)

    with rasterio.open(ortho_image_path) as src:
        # 영상 전체를 1024x1024 타일로 깔끔하게 나누기 위해 가로/세로 크기를 1024의 배수로 조정합니다.
        width = (src.width // tile_size) * tile_size
        height = (src.height // tile_size) * tile_size
        print(f"전체 영상 크기: ({src.width}, {src.height}), 조정된 크기: ({width}, {height})")

        # 만약 두 데이터의 좌표계(CRS)가 다르다면, 건물 데이터를 정사영상 기준으로 통일합니다.
        if gdf_buildings.crs != src.crs:
            gdf_buildings = gdf_buildings.to_crs(src.crs)

        # 생성된 파일 개수를 세기 위한 카운터입니다.
        tile_count = 0
        # 이중 for문을 사용해 전체 영상을 1024x1024 크기의 타일(window) 단위로 순회합니다.
        for j in range(0, height, tile_size):
            for i in range(0, width, tile_size):
                window = Window(i, j, tile_size, tile_size)
                tile_transform = src.window_transform(window)

                # 현재 타일의 지리적 좌표 경계를 계산하고, 이 경계와 겹치는 건물만 빠르게 찾아냅니다.
                tile_bounds = rasterio.windows.bounds(window, src.transform)
                tile_bbox = box(*tile_bounds)
                intersecting_buildings = gdf_buildings[gdf_buildings.intersects(tile_bbox)]

                # 현재 타일 안에 건물이 하나라도 있을 경우에만 데이터 생성 및 저장을 진행합니다.
                if not intersecting_buildings.empty:
                    # 1. '통합 라벨 마스크'를 메모리 상에서 생성합니다. (배경=0, 건물=1, 2, 3...)
                    simple_ids = range(1, len(intersecting_buildings) + 1)
                    shapes = [(geom, id) for geom, id in zip(intersecting_buildings.geometry, simple_ids)]

                    label_mask = rasterize(
                        shapes=shapes,
                        out_shape=(tile_size, tile_size),
                        transform=tile_transform,
                        fill=0,
                        all_touched=True,
                        dtype=rasterio.int32
                    )

                    # 2. 위에서 생성된 '통합 라벨 마스크'를 즉시 'targets' 딕셔너리로 변환합니다.
                    target_dict = convert_to_target_format(torch.from_numpy(label_mask).long())

                    # 3. 최종 결과물인 이미지와 'targets' 딕셔너리를 1:1로 매칭되는 개별 파일로 즉시 저장합니다.
                    #    이 방식 덕분에 RAM 사용량이 급증하지 않아 안정적인 처리가 가능합니다.
                    image_tile = src.read(window=window)
                    if image_tile.shape[0] > 3:
                        image_tile = image_tile[:3, :, :]

                    image_tensor = torch.from_numpy(image_tile).float()

                    torch.save(image_tensor, os.path.join(img_tile_dir, f"tile_{tile_count}.pt"))
                    torch.save(target_dict, os.path.join(lbl_dict_dir, f"tile_{tile_count}.pt"))

                    # 파일 저장 후 카운터를 1 증가시킵니다.
                    tile_count += 1

        print(f"\n--- 최종 데이터셋 생성 완료 ---")
        print(f"총 {tile_count}개의 [이미지 파일]과 [라벨 딕셔너리 파일] 쌍을 생성했습니다.")
        print(f"이미지 저장 위치: {img_tile_dir}")
        print(f"라벨 딕셔너리 저장 위치: {lbl_dict_dir}")



# 3단계: 함수 실행


# --- Google Drive 경로 설정 ---
# QGIS에서 전처리한 원본 데이터 파일들의 경로입니다.
ORTHO_TIF_PATH = "/content/drive/MyDrive/gis_project/clipped_정사영상.tif"
BUILDING_SHP_PATH = "/content/drive/MyDrive/gis_project/clipped_buildings.shp"
# 최종 결과물이 저장될 폴더의 경로입니다.
OUTPUT_DIR = "/content/drive/MyDrive/gis_project/final_dataset_per_file"
# --------------------------------

# 위에서 정의한 최종 데이터셋 생성 함수를 실행합니다.
create_final_dataset_per_file(ORTHO_TIF_PATH, BUILDING_SHP_PATH, OUTPUT_DIR)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
전체 영상 크기: (8850, 11114), 조정된 크기: (8192, 10240)

--- 최종 데이터셋 생성 완료 ---
총 80개의 [이미지 파일]과 [라벨 딕셔너리 파일] 쌍을 생성했습니다.
이미지 저장 위치: /content/drive/MyDrive/gis_project/final_dataset_per_file/images
라벨 딕셔너리 저장 위치: /content/drive/MyDrive/gis_project/final_dataset_per_file/labels_dict
